In [2]:
#imported stuff
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import os
import matplotlib.image as mpimg # read image
import cv2
from PIL import Image, ImageDraw, ImageFont
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import torch.optim as optim
from torchvision import datasets, models, transforms

plt.ion()   # interactive mode

In [5]:
#captureing
cam = cv2.VideoCapture(0)
count = 0
timeF = 10  #视频帧计数间隔频率 

while(True):
    tf, frame = cam.read()
    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #upper_red = np.array([130,255,255])
    #lower_red = np.array([110,100,100])
    #mask = cv2.inRange(frame, lower_red, upper_red)
    #frame = cv2.bitwise_and(frame,frame, mask=mask)
    if(count%timeF == 0): #每隔timeF帧进行存储操作
        cv2.imwrite("/Users/max/Downloads/wy/frame1{}.png".format(count), frame)
    count += 1
    cv2.imshow('Single Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()

In [6]:
#functions for processing faces
def detectFaces(image_name):
    img = cv2.imread(image_name)
    face_cascade = cv2.CascadeClassifier('C:/Users/Max/Anaconda3/pkgs/opencv3-3.1.0-py35_0/Library/etc/haarcascades/haarcascade_frontalface_default.xml')
    if img.ndim == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img
    faces = face_cascade.detectMultiScale(gray,1.2,5)#检测窗口的大小
    result = []
    for (x,y,width,height) in faces:
        result.append((x,y,x+width,y+height))
    return result

def saveFacles(image, count):
    faces = detectFaces(image)
    if faces:
        #save_dir = 'image.split('.')[0]+"_faces"'
        save_dir = 'D:/faces'
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        #count = 0
        for (x1, y1, x2, y2) in faces:
            file_name = os.path.join(save_dir,str(count)+".jpg")
            Image.open(image).crop((x1,y1,x2,y2)).save(file_name)
            count += 1

In [ ]:
#only faces
dir = 'D:/DATA/1/'
count = 1
for file in os.listdir(dir):
    iamges = dir + file
    saveFacles(iamges, count)
    count += 1

In [ ]:
#load data
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomSizedCrop(64),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Scale(64),
        transforms.CenterCrop(64),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'D:/DATA/catdog/64'
dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
         for x in ['train', 'val']}
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=4,
                                               shuffle=True, num_workers=4)
                for x in ['train', 'val']}
dset_sizes = {x: len(dsets[x]) for x in ['train', 'val']}
dset_classes = dsets['train'].classes

test_x,test_y = next(iter(dset_loaders['val']))
test_x = Variable(test_x)

use_gpu = torch.cuda.is_available()

In [ ]:
print(len(dsets['train']))
print(len(dsets['val']))

In [ ]:
torch.manual_seed(1)    # reproducible

# Hyper Parameters
EPOCH = 60               # train the training data n times, to save time, we just train 1 epoch
BATCH_SIZE = 4
LR = 0.001

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dset_loaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[dset_classes[x] for x in classes])

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         # input shape (1, 28, 28)
            nn.Conv2d(
                in_channels=3,              # input height
                out_channels=64,            # n_filters
                kernel_size=3,              # filter size
                stride=1,                   # filter movement/step
                padding=1,                  # if want same width and length of this image after con2d, padding=(kernel_size-1)/2 if stride=1
            ),                              # output shape (16, 28, 28)
            nn.ReLU(),                      # activation
            nn.MaxPool2d(kernel_size=2),    # choose max value in 2x2 area, output shape (16, 14, 14)
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(64, 64, 3, 1, 1),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        self.conv3 = nn.Sequential(         
            nn.Conv2d(64, 128, 3, 1, 1),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        self.conv4 = nn.Sequential(         
            nn.Conv2d(128, 128, 3, 1, 1),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        self.fc = nn.Linear(128 * 2 * 2, 2)   # fully connected layer, output n classes

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = x.view(x.size(0), -1)           # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        output = self.fc1(x)
        return output, x    # return x for visualization


cnn = CNN()
print(cnn)  # net architecture

optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)   # optimize all cnn parameters
loss_func = nn.CrossEntropyLoss()                       # the target label is not one-hotted

# following function (plot_with_labels) is for visualization, can be ignored if not interested
from matplotlib import cm
try: from sklearn.manifold import TSNE; HAS_SK = True
except: HAS_SK = False; print('Please install sklearn for layer visualization')
def plot_with_labels(lowDWeights, labels):
    plt.cla()
    X, Y = lowDWeights[:, 0], lowDWeights[:, 1]
    for x, y, s in zip(X, Y, labels):
        c = cm.rainbow(int(255 * s / 9)); plt.text(x, y, s, backgroundcolor=c, fontsize=9)
    plt.xlim(X.min(), X.max()); plt.ylim(Y.min(), Y.max()); plt.title('Visualize last layer'); plt.show(); plt.pause(0.01)

#plt.ion()
# training and testing
for epoch in range(EPOCH):
    for step, (x, y) in enumerate(dset_loaders['train']):   # gives batch data, normalize x when iterate train_loader
        b_x = Variable(x)   # batch x
        b_y = Variable(y)   # batch y

        output = cnn(b_x)[0]               # cnn output
        loss = loss_func(output, b_y)   # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients

        if step % 50 == 0:
            test_output, last_layer = cnn(test_x)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            accuracy = sum(pred_y == test_y) / float(test_y.size(0))
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data[0], '| test accuracy: %.2f' % accuracy)
#            if HAS_SK:
                
                # Visualization of trained flatten layer (T-SNE)
#                tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
#                plot_only = 500
#                low_dim_embs = tsne.fit_transform(last_layer.data.numpy()[:plot_only, :])
#                labels = test_y.numpy()[:plot_only]
#                plot_with_labels(low_dim_embs, labels)
#plt.ioff()

# print 10 predictions from test data
test_output, _ = cnn(test_x[:10])
pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
print(pred_y, 'prediction number')
print(test_y[:10].numpy(), 'real number')

In [ ]:
#save net and parms
torch.save(cnn.state_dict(), 'params.pkl')

In [ ]:
#load net and test
model = CNN()
model.load_state_dict(torch.load('params.pkl'))

In [ ]:
#captureing
cam = cv2.VideoCapture(0)
timeF = 10  #视频帧计数间隔频率 

while(True):
    tf, frame = cam.read()
    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if(count%timeF == 0): #每隔timeF帧进行存储操作
        cv2.imwrite("/Users/max/Downloads/wy/00.png", frame)
    cv2.imshow('Single Frame', frame)
    image = cv2.imread("/Users/max/Downloads/wy/00.png")
    face = saveFacles(image)
    face = Variable(face)
    output, _ = model(face)
    pred_y = torch.max(output, 1)[1].data.numpy().squeeze()
    if pred_y == 1:
        print("tyl")
    if pred_y == 0:
        print("wy")
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()